In [5]:
import json
import typing
import requests
import pandas as pd

df = pd.read_csv("../data/interim/data.csv")
if 'references' in df.columns:
    df.drop(['references'], axis=1, inplace=True)
 
# open citations
api_url_oc = "https://opencitations.net/index/coci/api/v1/references/"

opencitations_result = []

for index, row in df.loc[:, ["doi"]].iterrows():
    
    references = []

    url = f"{api_url_oc}{row.doi}"

    # headers = {"authorization": "YOUR-OPENCITATIONS-ACCESS-TOKEN"}
    headers: typing.Dict[str, str] = {}
    ret = requests.get(url, headers=headers, timeout=300)
    try:
        items = json.loads(ret.text)
        for item in items:
            references.append(item["cited"])
    except json.decoder.JSONDecodeError:
        print(f"Error retrieving citations from OpenCitations for DOI: {row.doi}")
    if len(references) == 0:
        opencitations_result.append([])
    else:
        opencitations_result.append(references)

df.insert(loc=len(df.columns), column="references", value=opencitations_result)

# crossref

api_url_cr = "https://api.crossref.org/works/"

for index, row in df.loc[:, ["doi"]].iterrows():
    
    if df.at[index,"references"] == []:
        references = []

        url = f"{api_url_cr}{row.doi}"

        headers: typing.Dict[str, str] = {}
        ret = requests.get(url, headers=headers, timeout=300)
        try:
            items = json.loads(ret.text)
            if "reference" in items["message"].keys():
                for item in items["message"]["reference"]:
                    if "DOI" in item.keys():
                        print(item)
                        references.append(item["DOI"])
        except json.decoder.JSONDecodeError:
            print(f"Error retrieving citations from Crossref for DOI: {row.doi}")
        if len(references) != 0:
            df.at[index,"references"] = references

# add results to data.csv

df.to_csv("../data/interim/data.csv", index=False)

{'status': 'ok', 'message-type': 'work', 'message-version': '1.0.0', 'message': {'indexed': {'date-parts': [[2024, 2, 5]], 'date-time': '2024-02-05T08:54:34Z', 'timestamp': 1707123274647}, 'reference-count': 0, 'publisher': 'JSTOR', 'issue': '1', 'content-domain': {'domain': [], 'crossmark-restriction': False}, 'short-container-title': ['MIS Quarterly'], 'published-print': {'date-parts': [[2001, 3]]}, 'DOI': '10.2307/3250961', 'type': 'journal-article', 'created': {'date-parts': [[2007, 2, 2]], 'date-time': '2007-02-02T22:33:05Z', 'timestamp': 1170455585000}, 'page': '107', 'source': 'Crossref', 'is-referenced-by-count': 5113, 'title': ['Review: Knowledge Management and Knowledge Management Systems: Conceptual Foundations and Research Issues'], 'prefix': '10.2307', 'volume': '25', 'author': [{'given': 'Maryam', 'family': 'Alavi', 'sequence': 'first', 'affiliation': []}, {'given': 'Dorothy E.', 'family': 'Leidner', 'sequence': 'additional', 'affiliation': []}], 'member': '1121', 'contai